In [1]:
from pyelasticsearch.client import ElasticSearch
import json
import requests
import pandas as pd
import math
import numpy as np
import boto3

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
config = json.load(open('../config/configsql.json', 'r'))
aws_access_key_id=config['dev']['aws']['key']
aws_secret_access_key=config['dev']['aws']['secret']
region = 'us-east-1'
endpoint = config['dev']['endpoint']
bucketname=config['dev']['bucketname']['bucketname']
index_name1 = 'candidates_cv'
index_name2 = 'parsed_resume'
es = ElasticSearch(endpoint, port=443)
comprehend = boto3.client(service_name='comprehend',aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key,region_name = region)

# all dict

In [10]:
d_all = es.search(index = index_name1, query={
                                     "query": {"match_all": {}},
                                     "size":  5
                                     })

In [11]:
all_user = {}
for i in range(len(d_all['hits']['hits'])):
    user = d_all['hits']['hits'][i]['_source']
    id = d_all['hits']['hits'][i]['_id']
    all_user[id] = user

# parsed resume

In [18]:
def extract_information(user_record):
    resp = {}
    for element in ['education', 'summary', 'projects', 'experience']:
        if element in user_record:
            text = user_record[element]
            #print(element,len(text))
            entity = comprehend.detect_entities(Text=text, LanguageCode='en')
            keyword = comprehend.detect_key_phrases(Text=text, LanguageCode='en')
        else:
            entity = {}
            keyword = {}
        resp[element] = {"entities": entity, "keywords": keyword}
    return resp

In [20]:
extract_information(user)

{'education': {'entities': {'Entities': [{'BeginOffset': 14,
     'EndOffset': 31,
     'Score': 0.6539674997329712,
     'Text': 'CALIFORNIA IRVINE',
     'Type': 'PERSON'},
    {'BeginOffset': 41,
     'EndOffset': 43,
     'Score': 0.5160495638847351,
     'Text': 'CA',
     'Type': 'LOCATION'},
    {'BeginOffset': 69,
     'EndOffset': 72,
     'Score': 0.8359231352806091,
     'Text': '8 F',
     'Type': 'QUANTITY'},
    {'BeginOffset': 87,
     'EndOffset': 104,
     'Score': 0.6400299072265625,
     'Text': 'CALIFORNIA IRVINE',
     'Type': 'PERSON'},
    {'BeginOffset': 106,
     'EndOffset': 116,
     'Score': 0.6707934141159058,
     'Text': 'IRVINE, CA',
     'Type': 'LOCATION'}],
   'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
     'content-length': '507',
     'content-type': 'application/x-amz-json-1.1',
     'date': 'Tue, 03 Apr 2018 00:08:34 GMT',
     'x-amzn-requestid': '2697289e-36d3-11e8-bb31-e118196dfed7'},
    'HTTPStatusCode': 200,
    'Reques

In [16]:
def ASCII(s):
    if s is None:
        return ''
    s2 = ''
    for i in range(len(s)):
        x = ord(s[i])
        if x >= 32 and x < 128:
            s2 += s[i]
        else:
            s2 += ' '
    return s2

In [17]:
resp_all = {}
for id in all_user.keys():
    print(id)
    user = all_user[id]
    for key in user.keys():
        user[key] = ASCII(user[key])
        if len(user[key]) > 5000:
            user[key] = user[key][0:5000]
    result = extract_information(user)
    resp_all[id] = result

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_B. TolgaOztanResume.pdf.txt


AttributeError: 'dict' object has no attribute 'lower'

In [9]:
resp_all

{'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Arun Varghese Resume 2-15.pdf.txt': {'education': {'entities': {'Entities': [{'BeginOffset': 2,
      'EndOffset': 24,
      'Score': 0.9833179116249084,
      'Text': 'University of Michigan',
      'Type': 'ORGANIZATION'},
     {'BeginOffset': 26,
      'EndOffset': 35,
      'Score': 0.7626931071281433,
      'Text': 'Ann Arbor',
      'Type': 'PERSON'},
     {'BeginOffset': 36,
      'EndOffset': 40,
      'Score': 0.9603113532066345,
      'Text': '2014',
      'Type': 'DATE'},
     {'BeginOffset': 43,
      'EndOffset': 47,
      'Score': 0.9639178514480591,
      'Text': '2016',
      'Type': 'DATE'},
     {'BeginOffset': 179,
      'EndOffset': 202,
      'Score': 0.926624059677124,
      'Text': 'Ross School of Business',
      'Type': 'ORGANIZATION'},
     {'BeginOffset': 204,
      'EndOffset': 281,
      'Score': 0.9366147518157959,
      'Text': 'UM Department of Economics and Schoo

# update to elasticsearch

In [2056]:
for id in resp_all.keys():
    user = resp_all[id]
    result = es.index(index_name2,'parsed_resume',user, id = id )

In [154]:
top10 = es.search(index = index_name2, query={"query": {"match_all": {}}})

In [158]:
top10

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Updated_Resume_Ayan.pdf.txt',
    '_index': 'parsed_resume',
    '_score': 1.0,
    '_source': {'education': {'entities': {'Entities': [{'BeginOffset': 0,
         'EndOffset': 24,
         'Score': 0.9541812539100647,
         'Text': 'University of Washington',
         'Type': 'ORGANIZATION'},
        {'BeginOffset': 46,
         'EndOffset': 53,
         'Score': 0.8231958746910095,
         'Text': 'Seattle',
         'Type': 'LOCATION'},
        {'BeginOffset': 55,
         'EndOffset': 57,
         'Score': 0.9677696824073792,
         'Text': 'WA',
         'Type': 'LOCATION'},
        {'BeginOffset': 62,
         'EndOffset': 75,
         'Score': 0.8251442909240723,
         'Text': '2014 May 2016',
         'Type': 'DATE'},
        {'BeginOffset': 160,
         'EndOffset': 164,
         'Score': 0.98060

In [74]:
count100 = es.search(index = index_name2, query=
    {
      "_source": 'education.keywords.KeyPhrases.Text',
      "size": 0,
      "aggs": {
        "group_by_coll_conv": {
          "terms": {
            "field": "education.keywords.KeyPhrases.Text.keyword",
            "size" : 500
          },
          "aggs": {
            "count" : {
              "value_count" : 
                { 
                  "field" : "education.keywords.KeyPhrases.Text.keyword" 
                } 
            }
        }
    }
}})

In [75]:
l_100 = count100['aggregations']['group_by_coll_conv']['buckets']

In [2299]:
keys = []
for i in l_100:
    key = i['key']
    keys.append(key)

In [2300]:
doc_counts = []
for i in l_100:
    doc_count = i['doc_count']
    doc_counts.append(doc_count)

In [2301]:
values = []
for i in l_100:
    value = i['count']['value']
    values.append(value)

In [2302]:
df = pd.DataFrame(
    {'word': keys,
     'doc_count': doc_counts,
     'values': values
    })

In [2303]:
count100['hits']['total']

622

In [2304]:
df['idf'] = np.log(count100['hits']['total']/df['doc_count'])

In [2305]:
df['tf'] = df['values']/sum(df['values'])

In [2306]:
df['tf-idf'] = df['idf']*df['tf']
df.head()

,doc_count,values,word,idf,tf,tf-idf
0,198,11712,University,1.144673,0.031296,0.035824
1,170,7781,Science,1.297142,0.020792,0.026970
2,86,4486,GPA,1.978593,0.011987,0.023718
3,74,5846,Machine Learning,2.128875,0.015621,0.033256
4,67,3077,Computer Science,2.228247,0.008222,0.018321


In [2495]:
pd.options.display.max_rows = 9999

In [2312]:
df_s = df.sort_values(by=['tf','word'], ascending=[False, True])

# all source

In [3]:
sources = ['education', 'summary','experience','projects']

In [4]:
results = []
for element in sources:
    result = es.search(index = index_name2, query=
    {
      "_source": element + '.keywords.KeyPhrases.Text',
      "size": 0,
      "aggs": {
        "group_by_coll_conv": {
          "terms": {
            "field": element + ".keywords.KeyPhrases.Text.keyword",
            "size" : 500
          },
          "aggs": {
            "count" : {
              "value_count" : 
                { 
                  "field" : element + ".keywords.KeyPhrases.Text.keyword"
                } 
            }
        }
    }
}})
    results.append(result)

In [156]:
results

[{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
  'aggregations': {'group_by_coll_conv': {'buckets': [{'count': {'value': 11712},
      'doc_count': 198,
      'key': 'University'},
     {'count': {'value': 7781}, 'doc_count': 170, 'key': 'Science'},
     {'count': {'value': 4486}, 'doc_count': 86, 'key': 'GPA'},
     {'count': {'value': 5846}, 'doc_count': 74, 'key': 'Machine Learning'},
     {'count': {'value': 3077}, 'doc_count': 67, 'key': 'Computer Science'},
     {'count': {'value': 6111}, 'doc_count': 58, 'key': 'Python'},
     {'count': {'value': 2965}, 'doc_count': 57, 'key': 'Technology'},
     {'count': {'value': 6208}, 'doc_count': 55, 'key': 'data'},
     {'count': {'value': 1843}, 'doc_count': 50, 'key': 'Master'},
     {'count': {'value': 3711}, 'doc_count': 48, 'key': 'Engineering'},
     {'count': {'value': 3271}, 'doc_count': 46, 'key': 'Data Mining'},
     {'count': {'value': 3411}, 'doc_count': 46, 'key': 'Los Angeles'},
     {'count': {'value': 2524}, 'doc

In [5]:
results_all = []
for i in range(len(results)):
    result_all = results[i]['aggregations']['group_by_coll_conv']['buckets']
    results_all.append(result_all)

In [157]:
results_all

[[{'count': {'value': 11712}, 'doc_count': 198, 'key': 'University'},
  {'count': {'value': 7781}, 'doc_count': 170, 'key': 'Science'},
  {'count': {'value': 4486}, 'doc_count': 86, 'key': 'GPA'},
  {'count': {'value': 5846}, 'doc_count': 74, 'key': 'Machine Learning'},
  {'count': {'value': 3077}, 'doc_count': 67, 'key': 'Computer Science'},
  {'count': {'value': 6111}, 'doc_count': 58, 'key': 'Python'},
  {'count': {'value': 2965}, 'doc_count': 57, 'key': 'Technology'},
  {'count': {'value': 6208}, 'doc_count': 55, 'key': 'data'},
  {'count': {'value': 1843}, 'doc_count': 50, 'key': 'Master'},
  {'count': {'value': 3711}, 'doc_count': 48, 'key': 'Engineering'},
  {'count': {'value': 3271}, 'doc_count': 46, 'key': 'Data Mining'},
  {'count': {'value': 3411}, 'doc_count': 46, 'key': 'Los Angeles'},
  {'count': {'value': 2524}, 'doc_count': 41, 'key': 'Ph.D.'},
  {'count': {'value': 2479}, 'doc_count': 40, 'key': 'Mathematics'},
  {'count': {'value': 2218}, 'doc_count': 40, 'key': 'Stat

In [7]:
dfs = []
for j in range(len(sources)):
    source = sources[j]
    keys = []
    for i in results_all[j]:
        key = i['key']
        keys.append(key)
    doc_counts = []
    for i in results_all[j]:
        doc_count = i['doc_count']
        doc_counts.append(doc_count)
    values = []
    for i in results_all[j]:
        value = i['count']['value']
        values.append(value)
    for i in results_all[j]:
        idfs = np.log([results[0]['hits']['total']/x for x in doc_counts]).tolist()
    for i in results_all[j]:
        tf = [x/sum(values) for x in values]
    for i in result_all[j]:
        tf_idf = [a*b for a,b in zip(tf,idfs)]
    aa = pd.DataFrame(
        {'word': keys,
         'doc_count': doc_counts,
         'values': values,
         'source': source,
         'idf':idfs,
         'tf':tf,
         'tf_idf':tf_idf
        })
    dfs.append(aa)

df = pd.concat(dfs)
df.reset_index(drop=True,inplace = True)
df= df.sort_values(by=['tf','word'], ascending=[False, True])

In [8]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [9]:
df.sort_values('word')

,doc_count,idf,source,tf,tf_idf,values,word
1819,2,5.739793,projects,0.000934,0.005360,200,"""AVR Microcontrollers"
1820,2,5.739793,projects,0.000934,0.005360,200,"""Dynamical Systems"
1821,2,5.739793,projects,0.000934,0.005360,200,"""Linear and Nonlinear Kalman Filter"
1822,2,5.739793,projects,0.000934,0.005360,200,"""Linear and Nonlinear Kalman Filters"
1823,2,5.739793,projects,0.000934,0.005360,200,"""Signal Processing"
1824,2,5.739793,projects,0.000934,0.005360,200,"""Single-channel and Multi-channel Blind"
429,2,5.739793,education,0.001235,0.007086,462,#ED33C-0949
430,2,5.739793,education,0.001235,0.007086,462,#H33G-1475
431,2,5.739793,education,0.001235,0.007086,462,#H34D-03. 2 [13] Sara Sadri
432,2,5.739793,education,0.001235,0.007086,462,#H53J-1547


In [10]:
df2 = df

In [11]:
df2['word'] = [x.lower() for x in df2['word']]

In [12]:
df3 = pd.DataFrame(df2.groupby(['word'])['doc_count'].sum())

In [13]:
df3.columns = ['count']

In [14]:
df3 = df3.reset_index()

In [15]:
df3.head()

,word,count
0,"""avr microcontrollers",2
1,"""dynamical systems",2
2,"""linear and nonlinear kalman filter",2
3,"""linear and nonlinear kalman filters",2
4,"""signal processing",2


In [16]:
def num_there(s):
    return any(i.isdigit() for i in s)

In [17]:
word_list = df3['word'].tolist()

In [18]:
df_num = pd.DataFrame([num_there(i) for i in word_list])

In [19]:
df4 = pd.concat([df3,df_num], axis=1)

In [20]:
df_no_num = df4[df4.loc[:][0]==False]

In [21]:
df_no_num .reset_index(drop=True, inplace=True)

In [22]:
# ''.join(filter(str.isalnum, string))

In [34]:
def remove_spe_chara(string):

    return re.sub(r'\W+', ' ', string)

In [45]:
no_spec_list = df_no_num['word'].tolist()

In [46]:
df_no_spe = pd.DataFrame([remove_spe_chara(i) for i in no_spec_list])

In [47]:
df_no_spe=df_no_spe.rename(columns = {0:'word2'})

In [48]:
df_final = pd.concat([df_no_num,df_no_spe],axis=1)

In [ ]:
df_final = df_final.reset_index()

In [50]:
df_final = df_final.reset_index()

In [231]:
df_final.to_clipboard()

In [51]:
df_final.head()

,index,word,count,0,word2
0,0,"""avr microcontrollers",2,False,avr microcontrollers
1,1,"""dynamical systems",2,False,dynamical systems
2,2,"""linear and nonlinear kalman filter",2,False,linear and nonlinear kalman filter
3,3,"""linear and nonlinear kalman filters",2,False,linear and nonlinear kalman filters
4,4,"""signal processing",2,False,signal processing


In [55]:
df_skill = pd.read_csv('skillset.csv')

In [56]:
df_skill

,index,doc_count,word,is_approved
0,710,272,python,1
1,566,197,machine learning,1
2,120,154,analysis,1
3,104,124,algorithms,1
4,586,118,matlab,1
5,346,116,data mining,1
6,522,98,java,1
7,787,91,sql,1
8,816,81,svm,1
9,724,77,random forest,1


In [70]:
df_skill_app = df_skill.loc[df_skill['is_approved']==1]

In [71]:
df_skill_app = df_skill_app[['word','is_approved','doc_count']]

In [72]:
df_skill_app.head()

,word,is_approved,doc_count
0,python,1,272
1,machine learning,1,197
2,analysis,1,154
3,algorithms,1,124
4,matlab,1,118


In [164]:
d_all = es.search(index = index_name1, query={
                                     "query": {"match_all": {}},
                                     "size":  100
                                     })

In [165]:
emails = []
for i in range(len(d_all['hits']['hits'])):
    fields = list(d_all['hits']['hits'][i]['_source'].keys())
    if 'email' in fields:
        email = d_all['hits']['hits'][i]['_source']['email']
    else:
        email = ''
    emails.append(email)

In [170]:
list(d_all['hits']['hits'][0]['_source'].keys())

['name', 'education', 'experience']

In [ ]:
# save to parquet
# save to s3



In [2557]:
filename = 'terms.parquet'

In [ ]:
df.to_parquet(filename)

In [2559]:
s3.Bucket(bucketname).upload_file(filename,'sys/parsed-keywords_v1.parquet')